In [18]:
from settrade_v2 import Investor
import pandas as pd
import datetime
from cassandra.cluster import Cluster
import time

In [19]:
# 1. เชื่อมต่อ Database
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()


session.set_keyspace('stock_data')

In [20]:
# 4. Connect Settrade
investor = Investor(
    app_id="gULd8BfryKxwQTAk",                                 
    app_secret="AL4r/Pi4zuxXBaDlMRn8jhkoytkaY+3LJqM8AThhp/h3", 
    broker_id="SANDBOX",
    app_code="SANDBOX",
    is_auto_queue=False
)   
market = investor.MarketData() # เปลี่ยนจาก .Market() เป็น .MarketData()
print("เชื่อมต่อ Settrade API และเข้าถึง Market Data สำเร็จ!")

# 5. ทดลองดึงข้อมูล
symbol = "PTT"
candlestick = market.get_candlestick(
    symbol=symbol, 
    interval="1d", 
    limit=10
)
print(candlestick)



เชื่อมต่อ Settrade API และเข้าถึง Market Data สำเร็จ!
{'lastSequence': 0, 'time': [1765731600, 1765818000, 1765904400, 1765990800, 1766077200, 1766336400, 1766422800, 1766509200, 1766595600, 1766682000], 'open': [31.25, 31.5, 31.25, 31.25, 31.25, 31.75, 32.0, 31.75, 31.75, 31.5], 'high': [31.75, 31.75, 31.5, 31.5, 31.75, 32.0, 32.25, 32.0, 31.75, 31.75], 'low': [31.25, 31.25, 31.0, 31.0, 31.0, 31.5, 31.75, 31.5, 31.5, 31.25], 'close': [31.75, 31.25, 31.0, 31.25, 31.5, 31.75, 31.75, 31.75, 31.5, 31.5], 'volume': [31811966, 26561266, 71082325, 93531188, 68507371, 43774425, 55660625, 30927077, 20536806, 34084620], 'value': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [21]:
print(dir(investor))

['Derivatives', 'Equity', 'MarketData', 'RealtimeDataConnection', 'SANDBOX', 'SANDBOX_BROKER_ID', 'SANDBOX_ENV', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', '_app_code', '_app_id', '_app_secret', '_broker_id', '_ctx', '_is_auto_queue']


In [22]:
import time
from datetime import datetime
import pandas as pd

# 1. ฟังก์ชันดึง candlestick
def get_candlestick_data(symbol):
    try:
        symbol = symbol.strip().upper()
        res = market.get_candlestick(
            symbol=symbol,
            interval="1d",
            limit=1000
        )
        return res
    except Exception as e:
        print(f"❌ API Error {symbol}: {e}")
        return None

# 2. ฟังก์ชัน insert ข้อมูล (เปลี่ยน ticker -> symbol ใน SQL)
def insert_historical_data(symbol, res):
    if not res or 'time' not in res:
        print(f"⚠️ {symbol}: ไม่มีข้อมูลในระบบ Sandbox")
        return

    time_list = res['time']
    num_rows = len(time_list)
    
    for i in range(num_rows):
        try:
            timestamp = datetime.fromtimestamp(time_list[i])
            
            # แก้ไขจาก ticker เป็น symbol ตามชื่อคอลัมน์ใหม่
            session.execute("""
                INSERT INTO candlestick_data (
                    symbol, date, open, high, low, close, volume
                ) VALUES (%s, %s, %s, %s, %s, %s, %s)
            """, (
                symbol,                    
                timestamp,                 
                float(res['open'][i]),     
                float(res['high'][i]),     
                float(res['low'][i]),      
                float(res['close'][i]),    
                int(res['volume'][i])      
            ))
        except Exception as e:
            print(f"❌ {symbol} บรรทัดที่ {i} ผิดพลาด: {e}")
            return 

    print(f"✅ บันทึก {symbol} สำเร็จ {num_rows} แถว")

# 3. โหลดรายชื่อหุ้นและเริ่มรัน (Limit 100)
try:
    df = pd.read_excel("/Users/hilmanyusoh/Desktop/Stock-market-chart-pattern-recognition-using-deep-learning/stock_pattern/data/settrade1.xlsx")
    stock_list = df['หลักทรัพย์'].dropna().unique().tolist()
    if 'หลักทรัพย์' in stock_list:
        stock_list.remove('หลักทรัพย์')
    
    # เอาแค่ 100 ตัวแรก
    stock_list = stock_list[:100] 
    
    print(f"🔎 พบรายชื่อหุ้น {len(stock_list)} ตัว (Limit 100). เริ่มดึงข้อมูลลงตารางใหม่...")

    for symbol in stock_list:
        symbol = symbol.strip().upper()
        print(f"🚀 กำลังเริ่มดึง: {symbol}...", end=" ")
        
        res = get_candlestick_data(symbol)
        insert_historical_data(symbol, res)
        
        time.sleep(1) 

except Exception as e:
    print(f"❌ เกิดข้อผิดพลาด: {e}")

🔎 พบรายชื่อหุ้น 100 ตัว (Limit 100). เริ่มดึงข้อมูลลงตารางใหม่...
🚀 กำลังเริ่มดึง: 24CS... ❌ API Error 24CS: Symbol not found [24CS]
⚠️ 24CS: ไม่มีข้อมูลในระบบ Sandbox
🚀 กำลังเริ่มดึง: 2S... ✅ บันทึก 2S สำเร็จ 731 แถว
🚀 กำลังเริ่มดึง: 3BBIF... ✅ บันทึก 3BBIF สำเร็จ 731 แถว
🚀 กำลังเริ่มดึง: A... ✅ บันทึก A สำเร็จ 671 แถว
🚀 กำลังเริ่มดึง: A5... ✅ บันทึก A5 สำเร็จ 727 แถว
🚀 กำลังเริ่มดึง: AAI... ✅ บันทึก AAI สำเร็จ 731 แถว
🚀 กำลังเริ่มดึง: AAV... ✅ บันทึก AAV สำเร็จ 731 แถว
🚀 กำลังเริ่มดึง: ABM... ✅ บันทึก ABM สำเร็จ 727 แถว
🚀 กำลังเริ่มดึง: ACAP... ✅ บันทึก ACAP สำเร็จ 63 แถว
🚀 กำลังเริ่มดึง: ACC... ✅ บันทึก ACC สำเร็จ 730 แถว
🚀 กำลังเริ่มดึง: ACE... ✅ บันทึก ACE สำเร็จ 731 แถว
🚀 กำลังเริ่มดึง: ACG... ✅ บันทึก ACG สำเร็จ 726 แถว
🚀 กำลังเริ่มดึง: ADB... ✅ บันทึก ADB สำเร็จ 731 แถว
🚀 กำลังเริ่มดึง: ADD... ✅ บันทึก ADD สำเร็จ 730 แถว
🚀 กำลังเริ่มดึง: ADVANC... ✅ บันทึก ADVANC สำเร็จ 731 แถว
🚀 กำลังเริ่มดึง: ADVICE... ✅ บันทึก ADVICE สำเร็จ 463 แถว
🚀 กำลังเริ่มดึง: AE... ✅ บันทึก AE สำเร็จ 7

In [23]:
# ต้องใส่ ALLOW FILTERING เพราะเราไม่ได้ระบุช่วงเวลา (Clustering Key) ครบถ้วนในบางกรณี
query = "SELECT * FROM candlestick_data WHERE symbol = '3BBIF' LIMIT 10 ALLOW FILTERING;"
rows = session.execute(query)
df_3bb = pd.DataFrame(list(rows))

print("ข้อมูลหุ้น 3BBIF:")
print(df_3bb)

ข้อมูลหุ้น 3BBIF:
  symbol       date  close  high   low  open   volume
0  3BBIF 2022-12-27   8.05  8.05  8.00  8.05  1607162
1  3BBIF 2022-12-28   8.00  8.05  8.00  8.05  3249547
2  3BBIF 2022-12-29   8.05  8.05  8.00  8.05  3115565
3  3BBIF 2022-12-30   8.05  8.10  8.00  8.05  4872314
4  3BBIF 2023-01-03   8.10  8.10  8.05  8.05  1526825
5  3BBIF 2023-01-04   8.10  8.15  8.05  8.10  7335558
6  3BBIF 2023-01-05   8.10  8.15  8.10  8.15  2989943
7  3BBIF 2023-01-06   8.15  8.20  8.10  8.15  5245742
8  3BBIF 2023-01-09   8.25  8.30  8.15  8.20  9169647
9  3BBIF 2023-01-10   8.25  8.30  8.20  8.25  9016478
